In [17]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd
import numpy as np
import json

In [18]:
!ls

app.py
data_engineering.ipynb
data_engineering_updated.ipynb
flask_queries.ipynb
info_data
raw_data
README.md
refugees_and_conflict.sqlite
templates


In [19]:
engine = create_engine('sqlite:///refugees_and_conflict.sqlite')

In [20]:
Base = automap_base()

In [21]:
Base.prepare(engine, reflect = True)

In [22]:
Base.classes.keys()

['asylum', 'battle_deaths', 'info_table', 'origin']

In [23]:
session = Session(engine)

In [32]:
Asylum = Base.classes.asylum
BattleDeaths = Base.classes.battle_deaths
Origin = Base.classes.origin
InfoTable = Base.classes.info_table

In [25]:
samples_df = pd.read_sql_table('asylum', engine)
samples_df.head()

,id,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,1,Albania,41.0000,20.0,ALB,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Algeria,28.0000,3.0,DZA,0,34,257,800,1916,...,339,505,236,267,256,147,107,110,86,0
2,3,American Samoa,-14.3333,-170.0,ASM,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Andorra,42.5000,1.6,AND,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Angola,-12.5000,18.5,AGO,2558,526,2321,12054,3974,...,0,25,0,0,0,0,0,0,0,0


## Query Country Names

In [26]:
country_names = session.query(Asylum.country_name).all()
country_names = [x[0] for x in country_names]
country_names

['Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Grenada',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hun

## Query Info Table

In [27]:
samples_df = pd.read_sql_table('info_table', engine)
samples_df.head()

,id,country_name,gdp_YR2015,population_YR2016,asylum_YR2016,origin_YR2016
0,1,Afghanistan,19215562178.9798,34656,17980,2501410
1,2,Albania,11335264966.561,2876,0,11050
2,3,Algeria,165874330876.321,40606,86,3675
3,4,American Samoa,659000000,56,0,0
4,5,Andorra,2811489408.89431,77,0,0


In [48]:
first_row_samples = session.query(InfoTable).first()
first_row_samples.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x169d05c3240>,
 'asylum_YR2016': 17980,
 'country_name': 'Afghanistan',
 'gdp_YR2015': '19215562178.9798',
 'id': 1,
 'origin_YR2016': 2501410,
 'population_YR2016': '34656'}

In [49]:
info_table = session.query(InfoTable.country_name, InfoTable.gdp_YR2015, InfoTable.population_YR2016, 
                           InfoTable.asylum_YR2016, InfoTable.origin_YR2016).all()

In [50]:
info_table = pd.DataFrame(info_table)
info_table.head()

,country_name,gdp_YR2015,population_YR2016,asylum_YR2016,origin_YR2016
0,Afghanistan,19215562178.9798,34656,17980,2501410
1,Albania,11335264966.561,2876,0,11050
2,Algeria,165874330876.321,40606,86,3675
3,American Samoa,659000000,56,0,0
4,Andorra,2811489408.89431,77,0,0


In [51]:
info_table = info_table.set_index('country_name').to_dict('index')

In [52]:
info_table

{'Afghanistan': {'asylum_YR2016': 17980,
  'gdp_YR2015': '19215562178.9798',
  'origin_YR2016': 2501410,
  'population_YR2016': '34656'},
 'Albania': {'asylum_YR2016': 0,
  'gdp_YR2015': '11335264966.561',
  'origin_YR2016': 11050,
  'population_YR2016': '2876'},
 'Algeria': {'asylum_YR2016': 86,
  'gdp_YR2015': '165874330876.321',
  'origin_YR2016': 3675,
  'population_YR2016': '40606'},
 'American Samoa': {'asylum_YR2016': 0,
  'gdp_YR2015': '659000000',
  'origin_YR2016': 0,
  'population_YR2016': '56'},
 'Andorra': {'asylum_YR2016': 0,
  'gdp_YR2015': '2811489408.89431',
  'origin_YR2016': 0,
  'population_YR2016': '77'},
 'Angola': {'asylum_YR2016': 0,
  'gdp_YR2015': '102962245246.708',
  'origin_YR2016': 8363,
  'population_YR2016': '28813'},
 'Antigua and Barbuda': {'asylum_YR2016': 0,
  'gdp_YR2015': '1364863037.03704',
  'origin_YR2016': 77,
  'population_YR2016': '101'},
 'Argentina': {'asylum_YR2016': 0,
  'gdp_YR2015': '584711485367.267',
  'origin_YR2016': 115,
  'populat